In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
with tf.device('/GPU:0'):
    # '/cpu:0', '/GPU:0'
    x = tf.placeholder(tf.float32, [None, 5])
    w = tf.Variable(tf.zeros([5, 1]))
    y = tf.matmul(x, w)
    t = tf.placeholder(tf.float32, [None, 1])
    loss = tf.reduce_sum(tf.square(y - t))
    train_step = tf.train.AdamOptimizer().minimize(loss)

In [4]:
%%time
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    train_t = np.array([5.2, 5.7, 8.6, 14.9, 18.2, 20.4, 25.5, 26.4, 22.8, 17.5, 11.1, 6.6])
    train_t = train_t.reshape([12, 1])
    
    train_x = np.zeros([12, 5])
    for row, month in enumerate(range(1, 13)):
        for col, n in enumerate(range(0, 5)):
            train_x[row][col] = month ** n
    
    i = 0
    for _ in range(100000):
        i += 1
        sess.run(train_step, feed_dict={x: train_x, t: train_t})
        if i % 10000 == 0:
            loss_val = sess.run(loss, feed_dict={x: train_x, t: train_t})
            print ('Step: %d, Loss: %f' % (i, loss_val))
    w_val = sess.run(w)
    print(w_val)

Step: 10000, Loss: 31.015018
Step: 20000, Loss: 29.294701
Step: 30000, Loss: 28.025995
Step: 40000, Loss: 27.321125
Step: 50000, Loss: 25.772881
Step: 60000, Loss: 24.780525
Step: 70000, Loss: 23.842400
Step: 80000, Loss: 22.976669
Step: 90000, Loss: 22.178181
Step: 100000, Loss: 42.597778
[[ 3.76436234]
 [-1.58896291]
 [ 1.78508031]
 [-0.20104764]
 [ 0.00550816]]
Wall time: 1min 2s


### GPU GTX950m
```
Wall time: 4min 42s
Wall time: 50.9 s
```
### GPU GTX1080
```
Wall time: 2min 18s
Wall time: 1min 2s
```
### CPU
```
Wall time: 54 s
Wall time: 1min 6s
```
### CPU i7-6700K 4GHz
```
Wall time: 19.6 s
Wall time: 18.8 s
```

In [ ]:
def predict(x):
    result = 0.0
    for n in range(0, 5):
        result += w_val[n][0] * x ** n
    return result

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
font = {'family' : 'meiryo'}
fig = plt.figure()
subplot = fig.add_subplot(1, 1, 1)
subplot.set_xlim(1, 12)
subplot.scatter(range(1, 13), train_t)
linex = np.linspace(1, 12, 100)
liney = predict(linex)
subplot.plot(linex, liney)